In [6]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split


## Load des données

In [7]:
val karate_graph = sc.textFile("data/soc-karate/soc-karate.mtx")


karate_graph: org.apache.spark.rdd.RDD[String] = data/soc-karate/soc-karate.mtx MapPartitionsRDD[1] at textFile at <console>:38


## Preprocess

In [8]:
var df_karate = karate_graph.toDF.withColumn("id",monotonicallyIncreasingId)


df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint]


In [9]:
df_karate = df_karate.withColumn("rank", row_number().over(Window.orderBy("id")))

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [10]:
df_karate.show(30,false)

+--------------------------------------------------------------------------------+----------+----+
|value                                                                           |id        |rank|
+--------------------------------------------------------------------------------+----------+----+
|%%MatrixMarket matrix coordinate pattern symmetric                              |0         |1   |
|%-------------------------------------------------------------------------------|1         |2   |
|% UF Sparse Matrix Collection, Tim Davis                                        |2         |3   |
|% http://www.cise.ufl.edu/research/sparse/matrices/Newman/karate                |3         |4   |
|% name: Newman/karate                                                           |4         |5   |
|% [Karate club, from Wayne Zachary, 1977]                                       |5         |6   |
|% id: 2399                                                                      |6         |7   |
|% date: 1

In [11]:
df_karate = df_karate.filter(df_karate("rank")>24)

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [12]:
df_karate = df_karate.drop("id","rank")

df_karate: org.apache.spark.sql.DataFrame = [value: string]


In [13]:
val df_karate_splitted = df_karate.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

df_karate_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]


In [14]:
val rows: RDD[Row] = df_karate_splitted.rdd

rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[19] at rdd at <console>:39


## Creation du graphe

In [15]:
var a= rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong,1)}
var default_user=(0, 0)
var graph = Graph.fromEdges(a, default_user)

a: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[20] at map at <console>:39
default_user: (Int, Int) = (0,0)
graph: org.apache.spark.graphx.Graph[(Int, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@20f7a8df


In [16]:
graph.edges.collect()

res1: Array[org.apache.spark.graphx.Edge[Int]] = Array(Edge(2,1,1), Edge(3,1,1), Edge(3,2,1), Edge(4,1,1), Edge(4,2,1), Edge(4,3,1), Edge(5,1,1), Edge(6,1,1), Edge(7,1,1), Edge(7,5,1), Edge(7,6,1), Edge(8,1,1), Edge(8,2,1), Edge(8,3,1), Edge(8,4,1), Edge(9,1,1), Edge(9,3,1), Edge(10,3,1), Edge(11,1,1), Edge(11,5,1), Edge(11,6,1), Edge(12,1,1), Edge(13,1,1), Edge(13,4,1), Edge(14,1,1), Edge(14,2,1), Edge(14,3,1), Edge(14,4,1), Edge(17,6,1), Edge(17,7,1), Edge(18,1,1), Edge(18,2,1), Edge(20,1,1), Edge(20,2,1), Edge(22,1,1), Edge(22,2,1), Edge(26,24,1), Edge(26,25,1), Edge(28,3,1), Edge(28,24,1), Edge(28,25,1), Edge(29,3,1), Edge(30,24,1), Edge(30,27,1), Edge(31,2,1), Edge(31,9,1), Edge(32,1,1), Edge(32,25,1), Edge(32,26,1), Edge(32,29,1), Edge(33,3,1), Edge(33,9,1), Edge(33,15,1), Edge(33...


In [17]:
graph.vertices.collect()

res2: Array[(org.apache.spark.graphx.VertexId, (Int, Int))] = Array((20,(0,0)), (13,(0,0)), (19,(0,0)), (34,(0,0)), (15,(0,0)), (4,(0,0)), (21,(0,0)), (16,(0,0)), (22,(0,0)), (25,(0,0)), (28,(0,0)), (29,(0,0)), (11,(0,0)), (14,(0,0)), (32,(0,0)), (30,(0,0)), (24,(0,0)), (27,(0,0)), (33,(0,0)), (23,(0,0)), (1,(0,0)), (6,(0,0)), (17,(0,0)), (3,(0,0)), (7,(0,0)), (9,(0,0)), (8,(0,0)), (12,(0,0)), (18,(0,0)), (31,(0,0)), (26,(0,0)), (10,(0,0)), (5,(0,0)), (2,(0,0)))


In [18]:
val newgraph = graph.outerJoinVertices(graph.degrees)((index,_,deg) => (index,deg.get))

newgraph: org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@abba59d


In [23]:
newgraph.triplets.collect()

res4: Array[org.apache.spark.graphx.EdgeTriplet[(org.apache.spark.graphx.VertexId, Int),Int]] = Array(((2,(2,9)),(1,(1,16)),1), ((3,(3,10)),(1,(1,16)),1), ((3,(3,10)),(2,(2,9)),1), ((4,(4,6)),(1,(1,16)),1), ((4,(4,6)),(2,(2,9)),1), ((4,(4,6)),(3,(3,10)),1), ((5,(5,3)),(1,(1,16)),1), ((6,(6,4)),(1,(1,16)),1), ((7,(7,4)),(1,(1,16)),1), ((7,(7,4)),(5,(5,3)),1), ((7,(7,4)),(6,(6,4)),1), ((8,(8,4)),(1,(1,16)),1), ((8,(8,4)),(2,(2,9)),1), ((8,(8,4)),(3,(3,10)),1), ((8,(8,4)),(4,(4,6)),1), ((9,(9,5)),(1,(1,16)),1), ((9,(9,5)),(3,(3,10)),1), ((10,(10,2)),(3,(3,10)),1), ((11,(11,3)),(1,(1,16)),1), ((11,(11,3)),(5,(5,3)),1), ((11,(11,3)),(6,(6,4)),1), ((12,(12,1)),(1,(1,16)),1), ((13,(13,2)),(1,(1,16)),1), ((13,(13,2)),(4,(4,6)),1), ((14,(14,5)),(1,(1,16)),1), ((14,(14,5)),(2,(2,9)),1), ((14,(14,...


In [24]:
newgraph.vertices.collect()

res5: Array[(org.apache.spark.graphx.VertexId, (org.apache.spark.graphx.VertexId, Int))] = Array((20,(20,3)), (13,(13,2)), (19,(19,2)), (34,(34,17)), (15,(15,2)), (4,(4,6)), (21,(21,2)), (16,(16,2)), (22,(22,2)), (25,(25,3)), (28,(28,4)), (29,(29,3)), (11,(11,3)), (14,(14,5)), (32,(32,6)), (30,(30,4)), (24,(24,5)), (27,(27,2)), (33,(33,12)), (23,(23,2)), (1,(1,16)), (6,(6,4)), (17,(17,2)), (3,(3,10)), (7,(7,4)), (9,(9,5)), (8,(8,4)), (12,(12,1)), (18,(18,2)), (31,(31,4)), (26,(26,3)), (10,(10,2)), (5,(5,3)), (2,(2,9)))


In [114]:
def sigmaIn2(comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcAttr._1==comId && triplet.dstAttr._1==comId))
    return graphVar.count.toDouble
}

def kiin2(i:Long, comId : Long, graph : Graph[(Long, Int),Int]) : Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==i && triplet.dstAttr._1==comId) ||(triplet.srcAttr._1==comId && triplet.dstAttr._1==i)))
    return graphVar.count.toDouble
}

def sigmaTot2(comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1!=comId)||(triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId)))
    return graphVar.count.toDouble
}

sigmaIn2: (comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
kiin2: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
sigmaTot2: (comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


In [123]:
def sigmaIn1(i:Long, comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1==comId) &&
                                                    !(triplet.srcId==i || triplet.dstId==i)))
    return graphVar.count.toDouble

}


def sigmaTot1(i:Long, comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1!=comId)||(triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId) && !(triplet.srcId==i || triplet.dstId==i)))
    return graphVar.count.toDouble
}

sigmaIn1: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
sigmaTot1: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


In [124]:
def ki(i: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcId==i || triplet.dstId==i))
    return graphVar.count.toDouble
}

def kiin(i:Long, comId : Long, graph : Graph[(Long, Int),Int]) : Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==i && triplet.dstAttr._1==comId) ||(triplet.srcAttr._1==comId && triplet.dstAttr._1==i)))
    return graphVar.count.toDouble
}

ki: (i: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
kiin: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


In [125]:
def DQ(sigmain:Double, sigmatot:Double, kivar:Double, kiinvar:Double, m:Long, graph:Graph[(Long, Int),Int]):Double = {
    return (((sigmain+kiinvar)/(2*m) - scala.math.pow((sigmatot+kivar)/(2*m),2)) - (sigmain/(2*m) - scala.math.pow(sigmatot/(2*m),2) - scala.math.pow(kivar/(2*m),2)))
}

DQ: (sigmain: Double, sigmatot: Double, kivar: Double, kiinvar: Double, m: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


In [163]:
var sigmain1 = sigmaIn1(1,14,newgraph)
var sigmatot1 = sigmaTot1(1,14,newgraph)
var kivar = ki(1, newgraph)
var kiinvar = kiin2(1,14,newgraph)
var m=newgraph.edges.count()/2

sigmain1: Double = 0.0
sigmatot1: Double = 5.0
kivar: Double = 16.0
kiinvar: Double = 1.0
m: Long = 39


In [164]:
var dq1 = DQ(sigmain1, sigmatot1, kivar, kiinvar, m , newgraph)

dq1: Double = -0.013477975016436564


In [223]:
var a = 0
var setDq =Set(): Set[(Int,Double)]
for (a <- 2 to 34){
    var sigmaindist = sigmaIn2(a.toLong, newgraph)
    var sigmatotdist = sigmaTot2(a.toLong, newgraph)
    var dq2 = DQ(sigmaindist, sigmatotdist, kivar, kiinvar, m, newgraph)
    setDq+=((a,dq2))
}

a: Int = 0
setDq: Set[(Int, Double)] = Set((4,-0.018737672583826435), (16,0.002301117685733052), (11,-0.002958579881656806), (8,-0.008218277449046671), (34,-0.07659434582511504), (30,-0.008218277449046671), (22,0.002301117685733052), (17,0.002301117685733052), (5,-0.002958579881656806), (26,-0.002958579881656806), (20,-0.002958579881656806), (23,0.002301117685733052), (33,-0.05029585798816569), (6,-0.008218277449046671), (27,0.002301117685733052), (2,-0.03451676528599608), (25,-0.002958579881656806), (18,0.002301117685733052), (3,-0.03977646285338594), (9,-0.013477975016436564), (19,0.002301117685733052), (28,-0.008218277449046671), (21,0.002301117685733052), (7,-0.008218277449046671), (14,-0.013477975016436564), (31,-0.008218277449046671), (29,-0.002958579881656806), (15,0.002301117685...


In [224]:
def maxCom(setDq : Set[(Int, Double)]) : Int = {
    var max = -1000.0
    var ind = -1
    setDq.foreach(x => {
        if (x._2>max)
        {
            ind=x._1
            max=x._2
        }
    })
    return ind
}

maxCom: (setDq: Set[(Int, Double)])Int


In [225]:
maxCom(setDq)

res63: Int = 12


In [299]:
def mooveItoC(i:Int, comId : Int, graph:Graph[(Long, Int),Int]) : Graph[(Long, Int),Int] = {
    val newVertices = newgraph.mapVertices { case (id, attr) => if (id==i) (comId.toLong, attr._2) else attr }
    return newVertices
}

mooveItoC: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])org.apache.spark.graphx.Graph[(Long, Int),Int]
